# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [41]:
import glob
import os
import btrack
print('Btrack version', btrack.__version__)
import napari
print('Napari version', napari.__version__)
import numpy as np

from skimage.io import imread
from natsort import natsorted
from napari_animation import AnimationWidget
from tqdm import tqdm
import tifffile as tiff

Btrack version 0.4.1
Napari version 0.4.7


# Load experiment and position list

In [87]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
expt = expt_IDs[0]
print('Experiment ID', expt)
pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
print('Position list', pos_list)

Experiment ID MK0000
Position list ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'Pos17', 'Pos18', 'Pos19', 'Pos20', 'Pos21', 'Pos22', 'Pos23']


# Manually select one position 

In [110]:
pos = 'Pos15'

# Load masks

In [107]:
mask_path = os.path.join(root_folder, '{}/{}/{}_cp_masks'.format(expt, pos, pos))
masks = natsorted(glob.glob(os.path.join(mask_path, '*.png')))
masks

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000001_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000002_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000003_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000004_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_cp_masks/img_channel001_position015_time000000005_z000_cp_masks.png',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Mana

# Filter according to channel

In [91]:
gfp_masks = [file for file in files if 'channel001' in file]
rfp_masks = [file for file in masks if 'channel002' in file]

# Load corresponding fluorescence images

In [112]:
image_path = os.path.join(root_folder, '{}/{}/{}_raw'.format(expt, pos, pos))
images = natsorted(glob.glob(os.path.join(image_path, '*.tif')))
gfp_images = [im for im in images if 'channel001' in im]
rfp_images = [im for im in images if 'channel002' in im]
irfp_images = [im for im in images if 'channel003' in im]

In [113]:
irfp_images

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000000_z000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000001_z000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000002_z000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000003_z000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000004_z000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_raw/img_channel003_position015_time000000005_z00

# Create stack using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [94]:
def stack_generator(files):
    """Stack generator"""
    
    for filename in tqdm(files):
        img = imread(filename)
        yield img

In [114]:
gfp_mask_stack = stack_generator(gfp_masks)
rfp_mask_stack = stack_generator(rfp_masks)

gfp_image_stack = stack_generator(gfp_images)
rfp_image_stack = stack_generator(rfp_images)
irfp_image_stack = stack_generator(irfp_images)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

two channel option

In [96]:
gfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, gfp_image_stack,
properties = ('area', 'mean_intensity')
)
rfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    rfp_mask_stack, rfp_image_stack,
    properties = ('area', 'mean_intensity')
)
irfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, irfp_image_stack,
properties = ('area', 'mean_intensity')
)

[INFO][2021/05/21 04:50:06 PM] Localizing objects from segmentation...
[INFO][2021/05/21 04:50:06 PM] Found intensity_image data
[INFO][2021/05/21 04:50:06 PM] Calculating weighted centroids using intensity_image


  0%|          | 0/1014 [00:00<?, ?it/s]



  0%|          | 0/1014 [00:00<?, ?it/s]

  0%|          | 1/1014 [00:00<03:00,  5.62it/s]



  0%|          | 1/1014 [00:00<02:52,  5.86it/s]

  0%|          | 2/1014 [00:00<02:33,  6.57it/s]



  0%|          | 2/1014 [00:00<02:36,  6.47it/s]

  0%|          | 3/1014 [00:00<02:25,  6.93it/s]



  0%|          | 3/1014 [00:00<02:27,  6.86it/s]

  0%|          | 4/1014 [00:00<02:27,  6.84it/s]



  0%|          | 4/1014 [00:00<02:32,  6.63it/s]

  0%|          | 5/1014 [00:00<02:30,  6.70it/s]



  0%|          | 5/1014 [00:00<02:26,  6.89it/s]

  1%|          | 6/1014 [00:00<02:23,  7.04it/s]



  1%|          | 6/1014 [00:00<02:24,  6.98it/s]

  1%|          | 7/1014 [00:01<02:22,  7.07it/s]



  1%|          | 7/1014 [00:01<02:2

  6%|▋         | 65/1014 [00:09<02:12,  7.14it/s]

  7%|▋         | 66/1014 [00:09<02:20,  6.75it/s]



  7%|▋         | 66/1014 [00:09<02:22,  6.63it/s]

  7%|▋         | 67/1014 [00:09<02:21,  6.69it/s]



  7%|▋         | 67/1014 [00:09<02:17,  6.88it/s]

  7%|▋         | 68/1014 [00:09<02:17,  6.90it/s]



  7%|▋         | 68/1014 [00:09<02:13,  7.08it/s]

  7%|▋         | 69/1014 [00:09<02:10,  7.23it/s]



  7%|▋         | 69/1014 [00:09<02:11,  7.19it/s]

  7%|▋         | 70/1014 [00:09<02:09,  7.28it/s]



  7%|▋         | 70/1014 [00:09<02:14,  7.00it/s]

  7%|▋         | 71/1014 [00:10<02:17,  6.87it/s]



  7%|▋         | 71/1014 [00:10<02:17,  6.87it/s]

  7%|▋         | 72/1014 [00:10<02:15,  6.97it/s]



  7%|▋         | 72/1014 [00:10<02:10,  7.22it/s]

  7%|▋         | 73/1014 [00:10<02:11,  7.14it/s]



  7%|▋         | 73/1014 [00:10<02:10,  7.24it/s]

  7%|▋         | 74/1014 [00:10<02:22,  6.59it/s]



  7%|▋         | 74/1014 [00:10<02:25,  6.47it/s]

  7%|▋       

 13%|█▎        | 132/1014 [00:18<02:11,  6.72it/s]



 13%|█▎        | 132/1014 [00:18<02:07,  6.94it/s]

 13%|█▎        | 133/1014 [00:18<02:05,  7.04it/s]



 13%|█▎        | 133/1014 [00:18<02:05,  7.01it/s]

 13%|█▎        | 134/1014 [00:19<02:05,  7.02it/s]



 13%|█▎        | 134/1014 [00:19<02:07,  6.92it/s]

 13%|█▎        | 135/1014 [00:19<02:05,  7.00it/s]



 13%|█▎        | 135/1014 [00:19<02:10,  6.72it/s]

 13%|█▎        | 136/1014 [00:19<02:10,  6.73it/s]



 13%|█▎        | 136/1014 [00:19<02:04,  7.05it/s]

 14%|█▎        | 137/1014 [00:19<02:06,  6.94it/s]



 14%|█▎        | 137/1014 [00:19<02:03,  7.11it/s]

 14%|█▎        | 138/1014 [00:19<02:01,  7.20it/s]



 14%|█▎        | 138/1014 [00:19<01:58,  7.37it/s]

 14%|█▎        | 139/1014 [00:19<02:04,  7.01it/s]



 14%|█▎        | 139/1014 [00:19<02:03,  7.11it/s]

 14%|█▍        | 140/1014 [00:19<02:00,  7.23it/s]



 14%|█▍        | 140/1014 [00:19<02:01,  7.22it/s]

 14%|█▍        | 141/1014 [00:20<01:59,  7.33i

 20%|█▉        | 198/1014 [00:28<01:57,  6.96it/s]



 20%|█▉        | 198/1014 [00:28<01:58,  6.86it/s]

 20%|█▉        | 199/1014 [00:28<01:56,  6.99it/s]



 20%|█▉        | 199/1014 [00:28<01:56,  7.00it/s]

 20%|█▉        | 200/1014 [00:28<01:53,  7.16it/s]



 20%|█▉        | 200/1014 [00:28<01:52,  7.22it/s]

 20%|█▉        | 201/1014 [00:28<01:54,  7.10it/s]



 20%|█▉        | 201/1014 [00:28<01:56,  6.97it/s]

 20%|█▉        | 202/1014 [00:29<01:54,  7.10it/s]



 20%|█▉        | 202/1014 [00:29<01:55,  7.01it/s]

 20%|██        | 203/1014 [00:29<01:54,  7.06it/s]



 20%|██        | 203/1014 [00:29<01:59,  6.78it/s]

 20%|██        | 204/1014 [00:29<02:00,  6.73it/s]



 20%|██        | 204/1014 [00:29<01:56,  6.95it/s]

 20%|██        | 205/1014 [00:29<01:55,  7.02it/s]



 20%|██        | 205/1014 [00:29<01:59,  6.77it/s]

 20%|██        | 206/1014 [00:29<01:59,  6.75it/s]



 20%|██        | 206/1014 [00:29<01:59,  6.77it/s]

 20%|██        | 207/1014 [00:29<01:58,  6.83i

 26%|██▌       | 264/1014 [00:38<01:46,  7.01it/s]

 26%|██▌       | 265/1014 [00:38<01:47,  6.99it/s]



 26%|██▌       | 265/1014 [00:38<01:45,  7.08it/s]

 26%|██▌       | 266/1014 [00:38<01:45,  7.08it/s]



 26%|██▌       | 266/1014 [00:38<01:43,  7.20it/s]

 26%|██▋       | 267/1014 [00:38<01:43,  7.25it/s]



 26%|██▋       | 267/1014 [00:38<01:42,  7.28it/s]

 26%|██▋       | 268/1014 [00:38<01:41,  7.35it/s]



 26%|██▋       | 268/1014 [00:38<01:42,  7.29it/s]

 27%|██▋       | 269/1014 [00:38<01:42,  7.30it/s]



 27%|██▋       | 269/1014 [00:38<01:40,  7.43it/s]

 27%|██▋       | 270/1014 [00:39<01:40,  7.41it/s]



 27%|██▋       | 270/1014 [00:39<01:39,  7.48it/s]

 27%|██▋       | 271/1014 [00:39<01:37,  7.60it/s]



 27%|██▋       | 271/1014 [00:39<01:42,  7.26it/s]

 27%|██▋       | 272/1014 [00:39<01:43,  7.14it/s]



 27%|██▋       | 272/1014 [00:39<01:45,  7.06it/s]

 27%|██▋       | 273/1014 [00:39<01:51,  6.64it/s]



 27%|██▋       | 273/1014 [00:39<01:49,  6.74i

...

 53%|█████▎    | 542/1014 [01:14<01:02,  7.51it/s]


 68%|██████▊   | 694/1014 [01:34<00:40,  7.81it/s]


 83%|████████▎ | 845/1014 [01:53<00:19,  8.46it/s]


 98%|█████████▊| 997/1014 [02:12<00:02,  7.68it/s]


100%|██████████| 1014/1014 [02:15<00:00,  7.50it/s]
[INFO][2021/05/21 04:55:06 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/21 04:55:06 PM] ...Found 10306 objects in 1014 frames.
[INFO][2021/05/21 04:55:06 PM] Localizing objects from segmentation...
[INFO][2021/05/21 04:55:06 PM] Found intensity_image data
[INFO][2021/05/21 04:55:06 PM] Calculating weighted centroids using intensity_image
[WARNING][2021/05/21 04:55:06 PM] ...Found no objects.


In [115]:
irfp_obj_from_generator = btrack.utils.segmentation_to_objects(
    gfp_mask_stack, irfp_image_stack,
properties = ('area', 'mean_intensity')
)

[INFO][2021/05/21 04:59:36 PM] Localizing objects from segmentation...
[INFO][2021/05/21 04:59:36 PM] Found intensity_image data
[INFO][2021/05/21 04:59:36 PM] Calculating weighted centroids using intensity_image
  0%|          | 0/1014 [00:00<?, ?it/s]

  0%|          | 1/1014 [00:00<03:15,  5.19it/s]

  0%|          | 2/1014 [00:00<02:48,  6.00it/s]

  0%|          | 3/1014 [00:00<02:24,  7.01it/s]

  0%|          | 4/1014 [00:00<02:32,  6.62it/s]

  0%|          | 5/1014 [00:00<02:33,  6.56it/s]

  1%|          | 6/1014 [00:00<02:34,  6.53it/s]

  1%|          | 7/1014 [00:01<02:25,  6.94it/s]

  1%|          | 8/1014 [00:01<02:27,  6.84it/s]

  1%|          | 9/1014 [00:01<02:24,  6.97it/s]

  1%|          | 10/1014 [00:01<02:21,  7.07it/s][A

  1%|          | 11/1014 [00:01<02:21,  7.08it/s]

  1%|          | 12/1014 [00:01<02:19,  7.18it/s]

  1%|▏         | 13/1014 [00:01<02:21,  7.07it/s]

  1%|▏         | 14/1014 [00:02<02:21,  7.08it/s]

  1%|▏         | 15/1014 [00:02<02:20,

 14%|█▎        | 139/1014 [00:19<02:01,  7.20it/s]

 14%|█▍        | 140/1014 [00:20<02:01,  7.18it/s]

 14%|█▍        | 141/1014 [00:20<02:04,  7.04it/s]

 14%|█▍        | 142/1014 [00:20<02:06,  6.90it/s]

 14%|█▍        | 143/1014 [00:20<02:05,  6.92it/s]

 14%|█▍        | 144/1014 [00:20<02:03,  7.03it/s]

 14%|█▍        | 145/1014 [00:20<02:06,  6.87it/s]

 14%|█▍        | 146/1014 [00:20<02:06,  6.88it/s]

 14%|█▍        | 147/1014 [00:21<02:07,  6.79it/s]

 15%|█▍        | 148/1014 [00:21<02:06,  6.86it/s]

 15%|█▍        | 149/1014 [00:21<02:08,  6.75it/s]

 15%|█▍        | 150/1014 [00:21<02:10,  6.63it/s]

 15%|█▍        | 151/1014 [00:21<02:05,  6.88it/s]

 15%|█▍        | 152/1014 [00:21<02:12,  6.50it/s]

 15%|█▌        | 153/1014 [00:21<02:17,  6.25it/s]

 15%|█▌        | 154/1014 [00:22<02:13,  6.42it/s]

 15%|█▌        | 155/1014 [00:22<02:09,  6.65it/s]

 15%|█▌        | 156/1014 [00:22<02:09,  6.61it/s]

 15%|█▌        | 157/1014 [00:22<02:11,  6.51it/s]

 16%|█▌     

 28%|██▊       | 280/1014 [00:41<01:53,  6.45it/s]

 28%|██▊       | 281/1014 [00:41<01:49,  6.71it/s]

 28%|██▊       | 282/1014 [00:41<01:52,  6.51it/s]

 28%|██▊       | 283/1014 [00:41<01:49,  6.70it/s]

 28%|██▊       | 284/1014 [00:41<01:43,  7.04it/s]

 28%|██▊       | 285/1014 [00:41<01:43,  7.07it/s]

 28%|██▊       | 286/1014 [00:41<01:42,  7.10it/s]

 28%|██▊       | 287/1014 [00:42<01:41,  7.16it/s]

 28%|██▊       | 288/1014 [00:42<01:41,  7.13it/s]

 29%|██▊       | 289/1014 [00:42<01:40,  7.19it/s]

 29%|██▊       | 290/1014 [00:42<01:40,  7.17it/s]

 29%|██▊       | 291/1014 [00:42<01:43,  7.00it/s]

 29%|██▉       | 292/1014 [00:42<01:44,  6.92it/s]

 29%|██▉       | 293/1014 [00:42<01:41,  7.09it/s]

 29%|██▉       | 294/1014 [00:43<01:45,  6.80it/s]

 29%|██▉       | 295/1014 [00:43<01:43,  6.93it/s]

 29%|██▉       | 296/1014 [00:43<01:40,  7.11it/s]

 29%|██▉       | 297/1014 [00:43<01:42,  7.02it/s]

 29%|██▉       | 298/1014 [00:43<01:44,  6.87it/s]

 29%|██▉    

 42%|████▏     | 421/1014 [01:02<01:26,  6.88it/s]

 42%|████▏     | 422/1014 [01:02<01:31,  6.49it/s]

 42%|████▏     | 423/1014 [01:02<01:36,  6.13it/s]

 42%|████▏     | 424/1014 [01:02<01:31,  6.48it/s]

 42%|████▏     | 425/1014 [01:02<01:28,  6.62it/s]

 42%|████▏     | 426/1014 [01:03<01:27,  6.68it/s]

 42%|████▏     | 427/1014 [01:03<01:24,  6.91it/s]

 42%|████▏     | 428/1014 [01:03<01:26,  6.76it/s]

 42%|████▏     | 429/1014 [01:03<02:09,  4.53it/s]

 42%|████▏     | 430/1014 [01:03<01:59,  4.87it/s]

 43%|████▎     | 431/1014 [01:04<01:52,  5.20it/s]

 43%|████▎     | 432/1014 [01:04<01:44,  5.58it/s]

 43%|████▎     | 433/1014 [01:04<01:37,  5.94it/s]

 43%|████▎     | 434/1014 [01:04<01:33,  6.22it/s]

 43%|████▎     | 435/1014 [01:04<01:27,  6.60it/s]

 43%|████▎     | 436/1014 [01:04<01:26,  6.68it/s]

 43%|████▎     | 437/1014 [01:05<01:30,  6.36it/s]

 43%|████▎     | 438/1014 [01:05<01:29,  6.46it/s]

 43%|████▎     | 439/1014 [01:05<01:27,  6.57it/s]

 43%|████▎  

 55%|█████▌    | 562/1014 [01:24<01:08,  6.58it/s]

 56%|█████▌    | 563/1014 [01:24<01:11,  6.32it/s]

 56%|█████▌    | 564/1014 [01:24<01:11,  6.31it/s]

 56%|█████▌    | 565/1014 [01:24<01:11,  6.29it/s]

 56%|█████▌    | 566/1014 [01:24<01:11,  6.26it/s]

 56%|█████▌    | 567/1014 [01:24<01:08,  6.52it/s]

 56%|█████▌    | 568/1014 [01:25<01:08,  6.47it/s]

 56%|█████▌    | 569/1014 [01:25<01:06,  6.70it/s]

 56%|█████▌    | 570/1014 [01:25<01:12,  6.16it/s]

 56%|█████▋    | 571/1014 [01:25<01:10,  6.27it/s]

 56%|█████▋    | 572/1014 [01:25<01:09,  6.34it/s]

 57%|█████▋    | 573/1014 [01:25<01:09,  6.31it/s]

 57%|█████▋    | 574/1014 [01:26<01:08,  6.41it/s]

 57%|█████▋    | 575/1014 [01:26<01:11,  6.16it/s]

 57%|█████▋    | 576/1014 [01:26<01:09,  6.29it/s]

 57%|█████▋    | 577/1014 [01:26<01:07,  6.44it/s]

 57%|█████▋    | 578/1014 [01:26<01:06,  6.52it/s]

 57%|█████▋    | 579/1014 [01:26<01:06,  6.58it/s]

 57%|█████▋    | 580/1014 [01:27<01:09,  6.26it/s]

 57%|█████▋ 

 69%|██████▉   | 703/1014 [01:46<00:53,  5.86it/s]

 69%|██████▉   | 704/1014 [01:47<00:53,  5.80it/s]

 70%|██████▉   | 705/1014 [01:47<00:51,  5.97it/s]

 70%|██████▉   | 706/1014 [01:47<00:52,  5.82it/s]

 70%|██████▉   | 707/1014 [01:47<00:52,  5.88it/s]

 70%|██████▉   | 708/1014 [01:47<00:50,  6.05it/s]

 70%|██████▉   | 709/1014 [01:47<00:51,  5.98it/s]

 70%|███████   | 710/1014 [01:48<00:52,  5.76it/s]

 70%|███████   | 711/1014 [01:48<00:51,  5.87it/s]

 70%|███████   | 712/1014 [01:48<00:53,  5.69it/s]

 70%|███████   | 713/1014 [01:48<00:51,  5.81it/s]

 70%|███████   | 714/1014 [01:48<00:52,  5.73it/s]

 71%|███████   | 715/1014 [01:48<00:52,  5.67it/s]

 71%|███████   | 716/1014 [01:49<00:54,  5.45it/s]

 71%|███████   | 717/1014 [01:49<00:54,  5.44it/s]

 71%|███████   | 718/1014 [01:49<00:54,  5.40it/s]

 71%|███████   | 719/1014 [01:49<00:52,  5.67it/s]

 71%|███████   | 720/1014 [01:49<00:49,  5.98it/s]

 71%|███████   | 721/1014 [01:50<00:50,  5.75it/s]

 71%|███████

 83%|████████▎ | 844/1014 [02:12<00:30,  5.50it/s]

 83%|████████▎ | 845/1014 [02:12<00:30,  5.59it/s]

 83%|████████▎ | 846/1014 [02:12<00:29,  5.73it/s]

 84%|████████▎ | 847/1014 [02:12<00:31,  5.37it/s]

 84%|████████▎ | 848/1014 [02:12<00:30,  5.44it/s]

 84%|████████▎ | 849/1014 [02:13<00:30,  5.49it/s]

 84%|████████▍ | 850/1014 [02:13<00:29,  5.56it/s]

 84%|████████▍ | 851/1014 [02:13<00:29,  5.55it/s]

 84%|████████▍ | 852/1014 [02:13<00:29,  5.53it/s]

 84%|████████▍ | 853/1014 [02:13<00:30,  5.36it/s]

 84%|████████▍ | 854/1014 [02:14<00:30,  5.33it/s]

 84%|████████▍ | 855/1014 [02:14<00:29,  5.35it/s]

 84%|████████▍ | 856/1014 [02:14<00:29,  5.29it/s]

 85%|████████▍ | 857/1014 [02:14<00:30,  5.11it/s]

 85%|████████▍ | 858/1014 [02:14<00:31,  4.88it/s]

 85%|████████▍ | 859/1014 [02:15<00:32,  4.77it/s]

 85%|████████▍ | 860/1014 [02:15<00:33,  4.65it/s]

 85%|████████▍ | 861/1014 [02:15<00:31,  4.91it/s]

 85%|████████▌ | 862/1014 [02:15<00:29,  5.18it/s]

 85%|███████

 97%|█████████▋| 985/1014 [02:39<00:05,  5.29it/s]

 97%|█████████▋| 986/1014 [02:39<00:05,  5.26it/s]

 97%|█████████▋| 987/1014 [02:39<00:05,  5.25it/s]

 97%|█████████▋| 988/1014 [02:39<00:05,  5.09it/s]

 98%|█████████▊| 989/1014 [02:39<00:04,  5.28it/s]

 98%|█████████▊| 990/1014 [02:40<00:04,  5.22it/s]

 98%|█████████▊| 991/1014 [02:40<00:04,  5.02it/s]

 98%|█████████▊| 992/1014 [02:40<00:04,  4.74it/s]

 98%|█████████▊| 993/1014 [02:40<00:04,  4.71it/s]

 98%|█████████▊| 994/1014 [02:41<00:04,  4.96it/s]

 98%|█████████▊| 995/1014 [02:41<00:04,  4.42it/s]

 98%|█████████▊| 996/1014 [02:41<00:03,  4.58it/s]

 98%|█████████▊| 997/1014 [02:41<00:03,  4.81it/s]

 98%|█████████▊| 998/1014 [02:41<00:03,  4.85it/s]

 99%|█████████▊| 999/1014 [02:42<00:03,  4.83it/s]

 99%|█████████▊| 1000/1014 [02:42<00:02,  4.76it/s][A

 99%|█████████▊| 1001/1014 [02:42<00:02,  4.61it/s]

 99%|█████████▉| 1002/1014 [02:42<00:02,  4.67it/s]

 99%|█████████▉| 1003/1014 [02:42<00:02,  4.72it/s]

 99%|█

In [116]:
# inspect the first object
irfp_obj_from_generator[0]

,ID,x,y,z,t,dummy,states,label,prob,area,mean_intensity
0,0,1088.977849,95.496319,0.0,0,False,0,5,0.0,1360,35.452206


## run btrack with the objects

We will use the objects from the generator here.

In [120]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    #tracker.append(gfp_obj_from_arr) #obj_from_generator)
    tracker.append(irfp_obj_from_generator)

    # set the volume
    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(root_folder,expt, pos, 'irfp_tracks.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
#     #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
    
#     gfp_tracks = tracker.tracks

    irfp_data, irfp_properties, irfp_graph = tracker.to_napari(ndim=2)
    
    irfp_tracks = tracker.tracks

[INFO][2021/05/21 05:05:26 PM] Loaded btrack: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/05/21 05:05:26 PM] btrack (v0.4.1) library imported
[INFO][2021/05/21 05:05:26 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/21 05:05:26 PM] Starting BayesianTracker session
[INFO][2021/05/21 05:05:26 PM] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json
[INFO][2021/05/21 05:05:26 PM] Loading motion model: b'MDCK_motion_Kristina'
[INFO][2021/05/21 05:05:26 PM] Setting max XYZ search radius to: 10
[INFO][2021/05/21 05:05:26 PM] Objects are of type: <class 'list'>
[INFO][2021/05/21 05:05:27 PM] Set volume to ((0, 1688), (0, 1352), (-100000.0, 100000.0))
[INFO][2021/05/21 05:05:27 PM] Starting tracking... 
[INFO][2021/05/21 05:05:27 PM] Tracking objects in frames 0 to 99 (of 1014)...
[INFO][2021/05/21 05:05:27 PM]  - Timing (Bayesian updates: 0.62ms, Linking: 0.17ms)
[INFO][202

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker usi#gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
    
#     gfp_tracks = tracker.tracksng a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_scribble_sparse.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    #tracker.append(rfp_obj_from_arr) #obj_from_generator)
    tracker.append(rfp_obj_from_generator)

    # set the volume
    tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(root_folder, pos, 'rfp_tracks.h5'), obj_type='obj_type_2')

    # get the tracks in a format for napari visualization
    rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
    
    rfp_tracks = tracker.tracks

# add image layers

### compile stacks if necessary

In [ ]:
### compile stacks first if necessary 
root_folder = os.path.dirname(PATH)
pos = os.path.basename(root_folder)
### make stack folder if necessary
output_dir = os.path.join(root_folder,pos+'_stacks')
os.mkdir(output_dir) if os.path.exists(output_dir) == False else print(output_dir, 'already exists')
### count number of channels
raw_file_dir = os.path.join(root_folder, pos+'_raw')
file_list = natsorted(os.listdir(raw_file_dir))
channels = list(set([fn.split('channel')[1][0:3] for fn in file_list]))
for channel in ['000', '001', '002']:#channels:
    channel_file_list = natsorted([fn for fn in os.listdir(raw_file_dir) if 'channel'+channel in fn])
    stack = []
    for file in tqdm(channel_file_list):
        file = os.path.join(raw_file_dir, file)
        tif = np.array(tiff.imread(file),dtype=np.uint8)
        ### crop??
#         tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
#         tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
        stack.append(tif)
    stack = np.stack(stack,axis=0)
    with tiff.TiffWriter(os.path.join(output_dir,'channel{}.tif'.format(channel)),imagej=True) as output_file:
        output_file.save(stack)
    print('channel',channel, 'saved out')
    

In [124]:
#image_path = output_dir#'/home/nathan/data/kraken/pcna/MK0000/Pos15/stacks'
image_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/{}/{}/{}_stacks'.format(expt, pos, pos)
image_paths = natsorted(glob.glob(os.path.join(image_path, '*.tif')))
image_paths

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel000.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel001.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel002.tif',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos15/Pos15_stacks/channel003.tif']

In [125]:
irfp = imread(image_paths[3])
rfp = imread(image_paths[2])

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [127]:
viewer = napari.Viewer()
# viewer.add_labels(gfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
#viewer.add_labels(rfp_stack)#, scale=(1., 1., 1.), name='Segmentation')
#viewer.add_tracks(gfp_data, properties=gfp_properties, graph=gfp_graph, name='GFP Tracks')
viewer.add_tracks(irfp_data)#, properties=rfp_properties, graph=rfp_graph, name='RFP Tracks')
# viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")
    
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')